<a href="https://colab.research.google.com/github/stankoj/lunar-ribosome/blob/main/ribosome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building neural networks using neural netowrks

Neuroevolution algorithms like [NEAT](https://neat-python.readthedocs.io/en/latest/) can be used to evolve an artifitial neural network to solve some simple task. But such algorithms do not scale well because mutating individual neurons does not have much impact on larger neural networks. It's true that we can control the number of mutations, but those mutations are not coherent and rarely make the neural network better.

DNA mutations that happen in real life on the other hand can result in changes on different layers of abstraction: a mutation can change a neuron, a group of neurons, or even higher lever features of the brain. Information seems to be encoded in a way to make code reuse possible, and to make mutations more likely to lead to positive change. All life on earth stores their "codebase" in form of DNA, and even the way information is encoded is always the same (at least on the lowest level - how DNA codons translate to amino acids).

But DNA would be nothing without something to parse it. Ribosomes have a big role in parsing the DNA information and converting it into proteins, which then interact with other things, which somehow leads to cells and organisms being built. DNA (the codebase) and ribosomes (the parser) must have somehow evolved in tandem, because they would not have any meaning wihout each other.

In this experiment, I want to explore if simulating an evolution of a codebase and a parser is possible, and if that could help make digital neuroevolution scale better. The codebase will be a random array of numbers, while the parser will be a neural network that is reading the codebase and outputing a new neural network to solve a particular task. The codebase (DNA) will be evolved by making random mutations to the array, while the parser (ribosome) will be evolved using python-NEAT. The task to solve will be the lunar lander OpenAI gym environment.

## The code

Let's first install python 3.8 because some dependancies we'll use are not compatible with newer versions. We will also install all dependancies here (this may take few minutes).


In [1]:
!sudo apt-get install python3.8 python3.8-distutils
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8
!sudo apt-get install python3-pip
!sudo apt-get install -y python-opengl ffmpeg
!sudo apt-get install --upgrade cmake
!sudo apt install swig xvfb
!sudo pip install pyvirtualdisplay ez_setup
!sudo pip install setuptools
!sudo pip install neat-python
!sudo pip install gymnasium
!sudo pip install swig
!sudo pip install "gymnasium[box2d]"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support
  python3.8-lib2to3 python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support python3.8
  python3.8-distutils python3.8-lib2to3 python3.8-minimal
0 upgraded, 8 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,422 kB of archives.
After this operation, 20.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.8-minimal amd64 3.8.20-1+jammy1 [796 kB]
Get:4 https://ppa.launchpadcontent.net

Let's start with writing the function that will generate our DNA. The first input parameter is the length of the DNA. The second one is the vocabulary, that is, what values a single element in the DNA code can have. Real DNA consists of 4 types of molecules, so we could use a vocabulary of size 4. But on the other hand, only groups of 3 molecules encode for some useful information (e.g. which amino acid to build, or where a gene starts/stops). Such a group of 3 molecules is called a codon. Since each codon has 3 molecules * 4 possilbe types of molecules, it means that a codon can encode for 64 different values, and we will use that number later on.

In [2]:
# Generate DNA
def generate_dna(length, vocab_size, seed = None):
    random.seed(seed)
    dna=[]
    for i in range(length):
        dna.append(random.randint(1, vocab_size))
    return dna

Now we need a fuction that will allow us to mutate the DNA. The following are some common mutations that happen to real DNA:


1. Point mutation - randomly change just a single DNA element
2. Duplication - duplicate a random portion of the DNA
3. Deletion - delete a random portion of the DNA
4. Translocation - copy a random porion of the DNA to a different location


In [3]:
# Mutate DNA
def mutate_dna(input_dna):
    #Pick mutation type
    mutation = random.randint(1, 4)

    # Copy DNA array because it's passed as reference
    dna = input_dna.copy()

    # Point mutation
    if (mutation == 1):
        # Pick random mutation location
        location = random.randint(0, len(dna) - 1)

        # Pick random new value
        mutated_value = random.randint(1, DNA_VOCAB_SIZE)

        # Mutate
        dna[location] = mutated_value

    # Duplication
    if (mutation == 2):
        # Pick random start location
        start = random.randint(0, len(dna) - 1)

        # Pick random end location
        stop = random.randint(start, len(dna) - 1)

        # Mutate
        dna = dna[:start] + dna[start:stop]*2 + dna[stop:]


    # Deletion
    if (mutation == 3):
        # Pick random start location
        start = random.randint(0, len(dna) - 1)

        # Pick random end location
        stop = random.randint(start, len(dna) - 1)

        # Mutate
        dna = dna[:start] + dna[stop:]

    # Translocation
    if (mutation == 4):
        # Pick random start location
        start = random.randint(0, len(dna) - 1)

        # Pick random end location
        stop = random.randint(start, len(dna) - 1)

        # Pick random translocation location
        translocation = random.randint(0, len(dna) - 1)

        # Mutate
        dna = dna[:translocation] + dna[start:stop] + dna[translocation:]

    return dna

NEAT requires a config file to run. We will create the config file next. Most values are left with default values, but the following 3 values were ajdusted:


1. fitness_threshold - this defines at which fitness the evolution should stop. For the lunar lander problem, it's usually considered that a fitenss of 200 is considered as a valid solution.
2. num_inputs - this should equal to the vocabulary size of our DNA.
3. num_outpus - this defines what actions the ribosome can perform based on the DNA it reads. That could be for example to create a neuron or a connection, or to jump to a different part of the DNA code. The exact number we use will be clearer when we build the ribosome fuction.



In [4]:
ribosome_config = '''[NEAT]
fitness_criterion     = max
fitness_threshold     = 200
pop_size              = 150
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = sigmoid
activation_mutate_rate  = 0.0
activation_options      = sigmoid

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.01

feed_forward            = False
initial_connection      = full_direct

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 0
num_inputs              = 64
num_outputs             = 11

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20
species_elitism      = 2

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2'''

with open("ribosome_config.tmp", "w") as text_file:
    text_file.write(ribosome_config)

As mentioned, the ribosome output will be a neural network - the brain. To simplify things, we will use NEAT to run that network. To be clear, we will not use NEAT to evolve the final network, but just to run it. Because of that, we need another config for that netowrk. Since we are not evolving the network, most of the config settings are irelevant.

In [5]:
brain_config = '''[NEAT]
fitness_criterion     = max
fitness_threshold     = 200
pop_size              = 150
reset_on_extinction   = False

[DefaultGenome]
# node activation options
activation_default      = sigmoid
activation_mutate_rate  = 0.0
activation_options      = sigmoid

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.0
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.01

feed_forward            = False
initial_connection      = unconnected

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 0
num_inputs              = 0
num_outputs             = 4

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 3.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20
species_elitism      = 2

[DefaultReproduction]
elitism            = 2
survival_threshold = 0.2'''

with open("brain_config.tmp", "w") as text_file:
    text_file.write(brain_config)

And the last part related to the configs is just a helper function that will be used to load them.

In [6]:
# Load NEAT config
def get_config(name):
    # Load config file
    local_dir = os.path.abspath('')
    config_file = os.path.join(local_dir, name)

    # Create config object
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                            neat.DefaultSpeciesSet, neat.DefaultStagnation,
                            config_file)
    return config

Now we are already at the main part of the code. It's the ribosome function that will take a DNA as input, and will output a neural network (the brain). The ribosome will take a DNA as input, and build a neural network from it. Everything is buit from scratch except the output nodes, which are hardcoded in this case to simplify some things.

In [7]:
# Convert DNA into NEAT genome
def ribosome(dna, ribosome_genome, ribosome_config, verbose = False):

    if (verbose == True):
      print("DNA PARSING PROCESS\n")
      print("Input connection IDs are -1 to -8, output connection IDs are 0-3, hidden connection IDs are 4 and above\n")

    # Create ribosome from genome
    ribosome = neat.nn.FeedForwardNetwork.create(ribosome_genome, ribosome_config)

    # Load config file
    brain_config = get_config('brain_config.tmp')

    # Set config parameters
    brain_config.genome_config.node_gene_type = DefaultNodeGene

    # Create blank genome
    brain_genome = neat.DefaultGenome(1)

    # Parse DNA and get output
    index = 0
    step = 0
    input_counter = 0

    # Create output nodes - output nodes are static
    for key in brain_config.genome_config.output_keys:
        brain_genome.nodes[key] = neat.DefaultGenome.create_node(brain_config.genome_config, key)
        brain_genome.nodes[key].bias = 0 # TODO: evolve this too

    while(True):
        # step counter
        if (step > MAX_STEPS):
            break
        if (index > len(dna) - 1):
            break

        # Prepare input as one-hot-encoded array
        input = [0] * DNA_VOCAB_SIZE
        input[dna[index]-1] = 1

        # Get outputs
        outputs = ribosome.activate(input)

        # Determine what the next step should be in neural net construction
        action = outputs.index(max(outputs[:6]))

        # Do nothing
        if (action == 0):
            if (verbose == True):
                print("DNA value: " + str(dna[index]) + " --> Action: do nothing")

        # Add hidden node
        if (action == 1):
            # Get new node ID
            new_node_id = brain_config.genome_config.get_new_node_key(brain_genome.nodes)

            # Create new node object
            node = neat.DefaultGenome.create_node(brain_config.genome_config, new_node_id)

            # Set bias
            node.bias = outputs[6]

            # Add node to gnome
            brain_genome.nodes[new_node_id] = node

            # Verbose outuput
            if (verbose == True):
                print("DNA value: " + str(dna[index]) + " --> Action: add hidden node")

        # Add input node
        if (action == 2 and input_counter < MAX_INPUTS):
            # Get new node ID
            input_counter += 1
            new_node_id = input_counter * (-1)

            # Create new node object
            node = neat.DefaultGenome.create_node(brain_config.genome_config, new_node_id)

            # # Set bias, but re-scale it from 0 - 1 to -1 - 1
            node.bias = outputs[7] * 2 - 1

            # Add node to gnome
            brain_genome.nodes[new_node_id] = node

            # Update input keys array in config
            brain_config.genome_config.input_keys.append(min(brain_config.genome_config.input_keys, default=0)-1)

            # Verbose output
            if (verbose == True):
                print("DNA value: " + str(dna[index]) + " --> Action: add input node")

        # Add connection
        if (action == 3):
            # Output 6 and 7 are between 0 and 1, but should be between min and max node id
            in_node = round(outputs[8] * (len(brain_genome.nodes)-1)) - input_counter

            # Same as above, but output nodes must have IDs above 0 based on NEAT python implementations
            out_node = round(outputs[9] * (len(brain_genome.nodes)-input_counter-1))

            # Set weight, but re-scale it from 0 - 1 to -1 - 1
            weight = int(outputs[10]) * 2 - 1

            # Add connection if it does not already exist
            if ((in_node, out_node) not in brain_genome.connections):
                brain_genome.add_connection(brain_config.genome_config, in_node, out_node, weight, True)

            # If exists
            else:
                brain_genome.connections[(in_node, out_node)].weight += weight

            # Verbose output
            if (verbose == True):
                print("DNA value: " + str(dna[index]) + " --> Action: add connection " + str(in_node) + " -> " + str(out_node) + " | weight: " + str(weight))

        # Move reading head back
        if (action == 4):
            if (index > 0):
                index = index - 2 # -2 to go one step back because we are always incrementing the index by 1 in every loop

            # Verbose output
            if (verbose == True):
                print("DNA value: " + str(dna[index]) + " --> Action: move reading head back")

        # Finish
        if (action == 5):

            # Verbose output
            if (verbose == True):
                print("DNA value: " + str(dna[index]) + " --> Action: stop parsing")

            break

        # Increment dna reading position and step counter
        index += 1
        step += 1

    return (brain_genome, brain_config)

Next comes a standard NEAT function that takes a population of ribosomes, and tests which has the best fitness.

In [8]:
# Evaluate fitness of provided genomes
def eval_genomes(ribosome_genomes, ribosome_config):

    global evolve_dna

    # Access global dna_pool
    global dna_pool

    # Deduplicate global DNA pool
    dna_pool.sort()
    dna_pool = list(dna_pool for dna_pool,_ in itertools.groupby(dna_pool))

    # Initialize new DNA pool every generation
    new_dna_pool = []

    # Loop through population
    for ribosome_genome_id, ribosome_genome in ribosome_genomes:

        # Randomize dna pool in case whole population has same fitness, to ensure different DNAs get carried over
        random.shuffle(dna_pool)

        for dna in dna_pool:
            brain, brain_config = ribosome(dna, ribosome_genome, ribosome_config)
            fitness = play(brain, brain_config)

            if (ribosome_genome.fitness is None or fitness > ribosome_genome.fitness):
                ribosome_genome.fitness = fitness
                ribosome_genome.dna = dna

        # Add best DNA and mutated versions to DNA pool
        new_dna_pool.append(ribosome_genome.dna)

        #if (evolve_dna == 1):
        if (evolve_dna == 1):
            new_dna_pool.append(mutate_dna(ribosome_genome.dna))

    # Save new DNA pool
    dna_pool = new_dna_pool

The play function will take one ribosome as input and will run a simulation to verify its fitness. We mainly rely on the reward system of the environment, but have also added some additional penalties for neural networks that are completely passive or that are in some way incomplete (e.g. no input neurons, no connections etc). We also use a flag evolve_dna to start mutating the DNA only after the parser/ribosome managed to build a non-passive brain.

In [9]:
# Do one run with the provided brain
def play(brain_genome, brain_config, render_mode = None):

    # Global flag to determine if dna should be mutated, or only ribosome
    global evolve_dna

    # Create neural network / brain from genome
    brain = neat.nn.FeedForwardNetwork.create(brain_genome, brain_config)

    passive = True

    # Create environment
    env = gym.make("LunarLander-v3", render_mode=render_mode)

    if (render_mode == 'rgb_array'):
      env = RecordVideo(env, './video')

    observation, info = env.reset(seed=42)

    # Trim observation depending on number of input neurons
    observation = observation[:len(brain_config.genome_config.input_keys)]

    # Run simulation and calculate fitness
    total_reward = 0
    for _ in range(1000):
        outputs = brain.activate(observation)
        action = outputs.index(max(outputs))

        # If outputs are not all zero, then start mutating dna too
        if (any(outputs) and action != 0):
            evolve_dna = 1
            passive = False

        observation, reward, terminated, truncated, info = env.step(action)

        # Trim observation depending on number of input neurons
        observation = observation[:len(brain_config.genome_config.input_keys)]

        total_reward += reward

        if terminated or truncated:
            env.close()
            break

    # Penalize brains that were completely idle
    if (passive == True):
        total_reward = -PASSIVE_PENALTY

    # Penalize brains that are incomplete (no inputs, or outputs or connections)
    if (len(brain_config.genome_config.input_keys) == 0):
        total_reward = total_reward - PASSIVE_PENALTY
    if (len(brain_config.genome_config.output_keys) == 0):
        total_reward = total_reward - PASSIVE_PENALTY
    if (len(brain_genome.connections) == 0):
        total_reward = total_reward - PASSIVE_PENALTY

    return total_reward

And finally, we have the main NEAT loop, which is the evolve function. It's task is to generate a population, and run the evolution until the fintess threshold is reached, or the max number of generations defined within it is reached.

In [10]:
# Run evolution
def evolve(ribosome_config):
    # Load configuration
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         ribosome_config)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Generate inital DNA
    dna_pool.append(generate_dna(INITIAL_DNA_LENGTH, DNA_VOCAB_SIZE))

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5))

    # Run for up to 300 generations.
    winner = p.run(eval_genomes, NUM_GENERATIONS)

    # Export winning ribosome genome, config and DNA
    save(winner, config, winner.dna)

Functions to export and import a genome, cinfig and DNA

In [11]:
# Export genome, config and DNA
def save(genome, config, dna):
    # Export genome
    with open("genome.pkl", "wb") as f:
        pickle.dump(genome, f)
        #f.close()

    # Export config
    with open("config.pkl", "wb") as f:
        pickle.dump(config, f)
        #f.close()

    # Export DNA
    with open("dna.pkl", "wb") as f:
        pickle.dump(dna, f)
        #f.close()

# Load genome, config and DNA
def load():
    # Load genome
    with open("genome.pkl", "rb") as f:
        genome = pickle.load(f)

    # Load config
    with open("config.pkl", "rb") as f:
        config = pickle.load(f)

    # Load DNA
    with open("dna.pkl", "rb") as f:
        dna = pickle.load(f)

    return genome, config, dna

What's left is to import some dependancies, define the constants/hyperparameters we used in the code, and run it.

In [12]:
import random
import os
import gymnasium as gym
import pickle
import neat
import itertools
from neat.genes import DefaultConnectionGene, DefaultNodeGene
from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo

# Settings
MAX_STEPS = 1000 # To prevent simulations running indefinitely
MAX_INPUTS = 8 # Max number of inputs the environment supports
NUM_GENERATIONS = 300 # For how many generations to run the evolution
DNA_VOCAB_SIZE = 64 # DNA vocab size
INITIAL_DNA_LENGTH = 100 # Initial DNA length
PASSIVE_PENALTY = 1000 # Penalty for passive or incomplete brains
evolve_dna = 0 # Flag for deciding when to start evolving DNA
dna_pool = []

The following code will start the evolution process. I have commented it out because it can take couple hours to finish, and you can skip that step and load a pre-trained ribosome and DNA in the following step. But if you want to evolve your own ribosome and DNA, then you can uncomment the code and run it (make sure to skip the next step in that case to avoid overwriting your trained ribosome and DNA). Initial generations will finish very quickly, but once the first non-passive brains emerge, generations will take much more time to finish.

In [13]:
# local_dir = os.path.abspath('')
# config_path = os.path.join(local_dir, 'ribosome_config.tmp')
# evolve(config_path)

Load pretrained ribosome (skip this step if you have evolved your own ribosome and DNA in the previous step).

In [14]:
!wget https://github.com/stankoj/lunar-ribosome/raw/refs/heads/main/config.pkl
!wget https://github.com/stankoj/lunar-ribosome/raw/refs/heads/main/dna.pkl
!wget https://github.com/stankoj/lunar-ribosome/raw/refs/heads/main/genome.pkl

--2024-12-05 23:32:20--  https://github.com/stankoj/lunar-ribosome/raw/refs/heads/main/config.pkl
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/stankoj/lunar-ribosome/refs/heads/main/config.pkl [following]
--2024-12-05 23:32:20--  https://raw.githubusercontent.com/stankoj/lunar-ribosome/refs/heads/main/config.pkl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4872 (4.8K) [application/octet-stream]
Saving to: ‘config.pkl’

config.pkl          100%[===================>]   4.76K  --.-KB/s    in 0s      

2024-12-05 23:32:20 (43.1 MB/s) - ‘config.pkl’ saved [4872/4872]

--2024-12-05 23:32:20--  h

Test out parsing of DNA in verbose mode. We could also try changing the DNA to see how that impacts the end result.

In [15]:
genome, config, dna = load()
ribosome(dna, genome, config, True)

DNA PARSING PROCESS

Input connection IDs are -1 to -8, output connection IDs are 0-3, hidden connection IDs are 4 and above

DNA value: 33 --> Action: add input node
DNA value: 6 --> Action: add connection -1 -> 0 | weight: -1
DNA value: 31 --> Action: add connection -1 -> 3 | weight: -1
DNA value: 62 --> Action: add connection -1 -> 2 | weight: -1
DNA value: 14 --> Action: add connection -1 -> 3 | weight: -1
DNA value: 19 --> Action: add connection -1 -> 3 | weight: -1
DNA value: 2 --> Action: add connection 2 -> 1 | weight: -1
DNA value: 62 --> Action: add connection -1 -> 2 | weight: -1
DNA value: 32 --> Action: add input node
DNA value: 4 --> Action: add connection -2 -> 3 | weight: -1
DNA value: 43 --> Action: add connection -2 -> 2 | weight: -1
DNA value: 45 --> Action: add connection -2 -> 3 | weight: -1
DNA value: 52 --> Action: add input node
DNA value: 52 --> Action: add input node
DNA value: 56 --> Action: add connection -4 -> 2 | weight: -1
DNA value: 33 --> Action: add in

(<neat.genome.DefaultGenome at 0x7df94d769b10>,
 <neat.config.Config at 0x7df94d94a350>)

Run simulation

In [16]:
# Jupyter rendering code used from https://medium.com/@coldstart_coder/visually-rendering-python-gymnasium-in-jupyter-notebooks-4413e4087a0f

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

import io
import base64
from IPython import display
from IPython.display import HTML
import warnings

warnings.filterwarnings('ignore')

def embed_video(video_file):
    video_data = io.open(video_file, 'r+b').read()
    encoded_data = base64.b64encode(video_data)
    display.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded_data.decode('ascii'))))

from gymnasium.wrappers import RecordEpisodeStatistics, RecordVideo
final_nn, nn_config = ribosome(dna, genome, config)
play(final_nn, nn_config, "rgb_array")

embed_video("video/rl-video-episode-0.mp4")


## Takeaways

* The good news is that the DNA and the ribosome ended up working together in sync to create a neural network that could solve the lunar lander problem.
* The bad news is that we did not end up with some fancy encoding technique nor with some interesting neural network. Evolution will always use the shortest path possible, which in this case was a simple neural network without any hidden neurons. Even for the weights, the deafult values were good enough.
* To evolve complex brains, we need complex environments. But starting with an environment where the learning curve is too steep won't work either. We need the environment to evolve in complexity and always provide challanges and opportunities for brains to evolve.
* A very granular reward system is crucial to prevent evolution from getting stuck. The penalties we introduced for passive brains really helped in the initial phase of evolution (sometimes an active brain that acummulated a lot of negative reward is better than a fully passive brain that did nothing).
* Algorithms like NEAT usually grow just hidden neurons, while input and output neurons are static. That is a problem when the input is a picture, because the number of input neurons is in that case too large for evolution to handle at once. That is one reason why I wanted to allow the algorithm to grow the number of input neurons over time. That may not have been very important in this environment with just 8 input neurons, but might be important in other environments with a much larger number of inputs. But just growing the number of input neurons may not be enough, because for example in case of processing photos, a small initial number of inputs should not mean that we are taking just a few out of many available pixels, but should mean that we are taking the whole photo, but just in a much downsampled format.
* We did not implement any methods for DNA to cross over and exchange genetic information, which may limit the evolution process.
* And finally, providing the right building blocks to the ribosome function for generating a neural network (add neuron, add connection, jump etc.) is likely another key factor in making this approach work, because not everything that is turing complete is efficient in the same way.

